# Pytorch Baseline

**Notes**
- Do not forget to enable the GPU (TPU) for training
- You have to add `kaggle_l5kit` as utility script
- Parts of the code below is from the [official example](https://github.com/lyft/l5kit/blob/master/examples/agent_motion_prediction/agent_motion_prediction.ipynb)


In [ ]:
import numpy as np
import os
import torch

from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import resnet18
from tqdm import tqdm
from typing import Dict
import matplotlib.pyplot as plt


from l5kit.data import LocalDataManager, ChunkedDataset
from l5kit.dataset import AgentDataset, EgoDataset
from l5kit.evaluation import write_pred_csv
from l5kit.rasterization import build_rasterizer

from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points

In [ ]:
from torch.utils.data.dataset import Subset


In [ ]:
DIR_INPUT = "/kaggle/input/lyft-motion-prediction-autonomous-vehicles"

SINGLE_MODE_SUBMISSION = f"{DIR_INPUT}/single_mode_sample_submission.csv"
MULTI_MODE_SUBMISSION = f"{DIR_INPUT}/multi_mode_sample_submission.csv"

# Training notebook's output.
WEIGHT_FILE = None

# Config

In [ ]:
cfg = {
    'format_version': 4,
    'model_params': {
        'history_num_frames': 0,
        'history_step_size': 1,
        'history_delta_time': 0.1,
        'future_num_frames': 50,
        'future_step_size': 1,
        'future_delta_time': 0.1
    },
    
    'raster_params': {
        'raster_size': [300, 300], # This is size of the image. That is 300*300 pixel
        'pixel_size': [0.5, 0.5], # One pixel corresponds to these many meters. 
                                    #  So, along the width and height of the image, one pixel will correspond to 0.5 metres in the real world.
        'ego_center': [0.25, 0.5], 
        'map_type': 'py_semantic', # The rasterizer to be used for visualization: 
        'satellite_map_key': 'aerial_map/aerial_map.png',
        'semantic_map_key': 'semantic_map/semantic_map.pb',
        'dataset_meta_key': 'meta.json',
        'filter_agents_threshold': 0.5
    },
    'train_data_loader': {
        'key': 'scenes/train.zarr',
        'batch_size': 8,
        'shuffle': True,
        'num_workers': 4
    },
    'test_data_loader': {
        'key': 'scenes/test.zarr',
        'batch_size': 16,
        'shuffle': False,
        'num_workers': 4
    }

}

In [ ]:
# set env variable for data
os.environ["L5KIT_DATA_FOLDER"] = DIR_INPUT
dm = LocalDataManager(None)

# Visualize

In [ ]:
zarr_dataset = ChunkedDataset('../input/lyft-motion-prediction-autonomous-vehicles/scenes/sample.zarr')
zarr_dataset.open()


In [ ]:
rast = build_rasterizer(cfg, dm)


## Visulaize AV

`EgoDataset`, `AgentDataset`   and  will allow us to iterate over the autnomous vehicle annotations.

In [ ]:
dataset_ego = EgoDataset(cfg, zarr_dataset, rast)

In [ ]:
data = dataset_ego[0]

im = data["image"].transpose(1, 2, 0)
im = dataset_ego.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)

plt.imshow(im[::-1])
plt.show()

This image contain:
* The green box: It is the autnomous vehicle.
* The pink line: The trajectory of the automous vehicle.
* The blue box: An agent.
* The yellow lines: The semantic maps.
* But what are the orange outlined boxes: Most prbably traffic posts, but not veru sure.

## Visualize Agent

In [ ]:
dataset_agent = AgentDataset(cfg, zarr_dataset, rast)
data = dataset_agent[0]

im = data["image"].transpose(1, 2, 0)
im = dataset_agent.rasterizer.to_rgb(im)
target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)

plt.imshow(im[::-1])
plt.show()

## Visualize satellite image

In [ ]:
# map_type was changed from 'py_semantic' to 'py_satellite'.
cfg["raster_params"]["map_type"] = "py_satellite"
satellite_rasterizer = build_rasterizer(cfg, dm)
satellite_dataset = EgoDataset(cfg, zarr_dataset, satellite_rasterizer)

data = satellite_dataset[0]
im = data["image"].transpose(1, 2, 0)
im = dataset_agent.rasterizer.to_rgb(im)
plt.imshow(im[::-1])
plt.show()

## Dataset, dataloader

Change config to load train data.

Use 10 frames history for model

In [ ]:
cfg["raster_params"]["map_type"] = "py_semantic"
cfg["model_params"]["history_num_frames"] = 10


In [ ]:
# ===== INIT DATASET
train_cfg = cfg["train_data_loader"]

# Rasterizer
rasterizer = build_rasterizer(cfg, dm)

# Test dataset/dataloader
train_zarr = ChunkedDataset(dm.require(train_cfg["key"])).open()
# train_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
train_dataset = AgentDataset(cfg, train_zarr, rasterizer)

train_dataset = Subset(train_dataset, np.arange(100))

train_dataloader = DataLoader(train_dataset,
                             shuffle=train_cfg["shuffle"],
                             batch_size=train_cfg["batch_size"],
                             num_workers=train_cfg["num_workers"])

In [ ]:
# ===== INIT DATASET
test_cfg = cfg["test_data_loader"]

# Rasterizer
# rasterizer = build_rasterizer(cfg, dm)

# Test dataset/dataloader
test_zarr = ChunkedDataset(dm.require(test_cfg["key"])).open()
test_mask = np.load(f"{DIR_INPUT}/scenes/mask.npz")["arr_0"]
test_dataset = AgentDataset(cfg, test_zarr, rasterizer, agents_mask=test_mask)

# test_dataset = Subset(test_dataset, np.arange(100))
test_dataloader = DataLoader(test_dataset,
                             shuffle=test_cfg["shuffle"],
                             batch_size=test_cfg["batch_size"],
                             num_workers=test_cfg["num_workers"])


print(test_dataloader)

Shape of input image

In [ ]:
test_dataset[0]['image'].shape

In [ ]:
test_dataset[101]['track_id']

In [ ]:
test_dataset[0]['target_positions'].shape

## Model

Model:
* input : channel * image_size * image_size . Ex with 10 history num frame and image size = 300*300. Input = (25*300*300)
* output: num_node + num_node * 2 * future_num_frames . Ex with 3-mode and 50 frame predict. Output have shape = (303,)

In [ ]:
class LyftModel(nn.Module):
    
    def __init__(self, cfg: Dict,num_modes = 3):
        super().__init__()
        
        self.backbone = resnet18(pretrained=False)
        
        num_history_channels = (cfg["model_params"]["history_num_frames"] + 1) * 2
        num_in_channels = 3 + num_history_channels

        self.backbone.conv1 = nn.Conv2d(
            num_in_channels,
            self.backbone.conv1.out_channels,
            kernel_size=self.backbone.conv1.kernel_size,
            stride=self.backbone.conv1.stride,
            padding=self.backbone.conv1.padding,
            bias=False,
        )
        
        # This is 512 for resnet18 and resnet34;
        # And it is 2048 for the other resnets
        backbone_out_features = 512

        # X, Y coords for the future positions (output shape: Bx50x2)
        self.future_len = cfg["model_params"]["future_num_frames"]
        num_targets = 2 * self.future_len

        # You can add more layers here.
        self.head = nn.Sequential(
            # nn.Dropout(0.2),
            nn.Linear(in_features=backbone_out_features, out_features=4096),
        )
        self.num_preds = num_targets * num_modes
        self.num_modes = num_modes
        self.logit = nn.Linear(4096, out_features=self.num_preds+self.num_modes)
        
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        x = self.backbone.layer1(x)
        x = self.backbone.layer2(x)
        x = self.backbone.layer3(x)
        x = self.backbone.layer4(x)

        x = self.backbone.avgpool(x)
        x = torch.flatten(x, 1)
        
        x = self.head(x)
        x = self.logit(x)
        bs, _ = x.shape
        pred, confidences = torch.split(x, self.num_preds, dim=1)
        pred = pred.view(bs, self.num_modes, self.future_len, 2)
        assert confidences.shape == (bs, self.num_modes)
        confidences = torch.softmax(confidences, dim=1)
        return pred, confidences


In [ ]:
# ==== INIT MODEL
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = LyftModel(cfg,num_modes=3)
model.to(device)

if WEIGHT_FILE is not None:
    # Saved state dict from the training notebook
    model_state = torch.load(WEIGHT_FILE, map_location=device)
    model.load_state_dict(model_state['model_state_dict'])

# Training

## Loss function

In [ ]:
# --- Function utils ---
# Original code from https://github.com/lyft/l5kit/blob/20ab033c01610d711c3d36e1963ecec86e8b85b6/l5kit/l5kit/evaluation/metrics.py
import numpy as np

import torch
from torch import Tensor


def pytorch_neg_multi_log_likelihood_batch(
    gt: Tensor, pred: Tensor, confidences: Tensor, avails: Tensor
) -> Tensor:
    """
    Compute a negative log-likelihood for the multi-modal scenario.
    log-sum-exp trick is used here to avoid underflow and overflow, For more information about it see:
    https://en.wikipedia.org/wiki/LogSumExp#log-sum-exp_trick_for_log-domain_calculations
    https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/
    https://leimao.github.io/blog/LogSumExp/
    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(modes)x(time)x(2D coords)
        confidences (Tensor): array of shape (bs)x(modes) with a confidence for each mode in each sample
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    assert len(pred.shape) == 4, f"expected 3D (MxTxC) array for pred, got {pred.shape}"
    batch_size, num_modes, future_len, num_coords = pred.shape

    assert gt.shape == (batch_size, future_len, num_coords), f"expected 2D (Time x Coords) array for gt, got {gt.shape}"
    assert confidences.shape == (batch_size, num_modes), f"expected 1D (Modes) array for gt, got {confidences.shape}"
    assert torch.allclose(torch.sum(confidences, dim=1), confidences.new_ones((batch_size,))), "confidences should sum to 1"
    assert avails.shape == (batch_size, future_len), f"expected 1D (Time) array for gt, got {avails.shape}"
    # assert all data are valid
    assert torch.isfinite(pred).all(), "invalid value found in pred"
    assert torch.isfinite(gt).all(), "invalid value found in gt"
    assert torch.isfinite(confidences).all(), "invalid value found in confidences"
    assert torch.isfinite(avails).all(), "invalid value found in avails"

    # convert to (batch_size, num_modes, future_len, num_coords)
    gt = torch.unsqueeze(gt, 1)  # add modes
    avails = avails[:, None, :, None]  # add modes and cords

    # error (batch_size, num_modes, future_len)
    error = torch.sum(((gt - pred) * avails) ** 2, dim=-1)  # reduce coords and use availability

    with np.errstate(divide="ignore"):  # when confidence is 0 log goes to -inf, but we're fine with it
        # error (batch_size, num_modes)
        error = torch.log(confidences) - 0.5 * torch.sum(error, dim=-1)  # reduce time

    # use max aggregator on modes for numerical stability
    # error (batch_size, num_modes)
    max_value, _ = error.max(dim=1, keepdim=True)  # error are negative at this point, so max() gives the minimum one
    error = -torch.log(torch.sum(torch.exp(error - max_value), dim=-1, keepdim=True)) - max_value  # reduce modes
    # print("error", error)
    return torch.mean(error)


def pytorch_neg_multi_log_likelihood_single(
    gt: Tensor, pred: Tensor, avails: Tensor
) -> Tensor:
    """

    Args:
        gt (Tensor): array of shape (bs)x(time)x(2D coords)
        pred (Tensor): array of shape (bs)x(time)x(2D coords)
        avails (Tensor): array of shape (bs)x(time) with the availability for each gt timestep
    Returns:
        Tensor: negative log-likelihood for this example, a single float number
    """
    # pred (bs)x(time)x(2D coords) --> (bs)x(mode=1)x(time)x(2D coords)
    # create confidence (bs)x(mode=1)
    batch_size, future_len, num_coords = pred.shape
    confidences = pred.new_ones((batch_size, 1))
    return pytorch_neg_multi_log_likelihood_batch(gt, pred.unsqueeze(1), confidences, avails)

## Train

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [ ]:
model.train()
for epoch in range(1):
    running_loss = 0.0
    for data in tqdm(train_dataloader):
        inputs = data["image"].to(device)
        target_availabilities = data["target_availabilities"].to(device)
        targets = data["target_positions"].to(device)
        optimizer.zero_grad()
        pred, confidences  = model(inputs)
        loss = pytorch_neg_multi_log_likelihood_batch(targets, pred, confidences, target_availabilities)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    print(running_loss)

# Predicting

In [ ]:
model.eval()

future_coords_offsets_pd = []
timestamps = []
agent_ids = []
confidences_list = []
with torch.no_grad():
    dataiter = tqdm(test_dataloader)
    
    for data in dataiter:

        inputs = data["image"].to(device)
        target_availabilities = data["target_availabilities"].unsqueeze(-1).to(device)
        targets = data["target_positions"].to(device)

        pred, confidences  = model(inputs)
        
        future_coords_offsets_pd.append(pred.cpu().numpy().copy())
        timestamps.append(data["timestamp"].numpy().copy())
        agent_ids.append(data["track_id"].numpy().copy())
        confidences_list.append(confidences.cpu().numpy().copy())

In [ ]:
write_pred_csv('submission.csv',
               timestamps=np.concatenate(timestamps),
               track_ids=np.concatenate(agent_ids),
               coords=np.concatenate(future_coords_offsets_pd),
              confs=np.concatenate(confidences_list)
              )